<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
#importing libraries
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import pydicom
import os
import sys
import skimage 

In [ ]:
files = [pydicom.dcmread(path + '/' + s) for s in sorted(os.listdir(path))] #Storing all images
images = [s.pixel_array for s in files]
dcm_gray = np.zeros([512,512],dtype = np.uint8)
gray_images = np.uint8([cv.normalize(s, dcm_gray, 0, 255, cv.NORM_MINMAX) for s in images])
dcm_equalized = [cv.equalizeHist(s) for s in gray_images]

In [ ]:
#Gaussian Filter bank
#images['GaussianBlur']['Image Number (1-100)'][]
images = {}
images['LOG'] = {}
images['DOG'] = {}
images['GaussianBlur'] = {}
images['Gray'] = {}
images['Gray_Equalized'] = {}

rt = 0
for i in gray_images:
    rt = rt+1
    
    #Gaussian Blur
    images['GaussianBlur'][rt] = {}
    for j in range(1,12):
        if j%2 == 0:
            pass
        else:
            gtemp = cv.GaussianBlur(i,(j,j),0)
            images['GaussianBlur'][rt][j] = gtemp

In [ ]:
#Laplacian Filter bank
#images['Laplacian']['Image number (1-100)'][kernel size of the blurred image(1-11)][kernel size of the Laplacian(1-31)]
ddepth = cv.CV_16S
n = len(gray_images)
for i in range(1,n+1):
    ker = -1
    images['LOG'][i] = {}
    for j in images['GaussianBlur'][i]:
        ker = ker + 2
        temp = images['GaussianBlur'][i][j]
        #plt.imshow(temp)
        #plt.show()
        images['LOG'][i][ker] = {}
        for k in range(1,32):
            if k%2 == 0:
                pass
            else:
                temp1 = images['GaussianBlur'][i][j]
                ltemp = np.uint8(cv.Laplacian(temp1,ddepth,ksize = k))
                images['LOG'][i][ker][k] = ltemp

In [ ]:
#images['DOG']['Set (set1 = sigma1 values of 1, 1.6 and set2 = sigma1 values of 10, 16)']['Image number (1-100)'][kernel size of first blurred image(1-11)][kernel size of second blurred image(1-11)]
#images['DOG'] = {}
rt = 0
for s in range(1,3):
    if s == 1:
        images['DOG'][s] = {}
        for i in gray_images:
            rt = rt+1
            images['DOG'][s][rt] = {}
            for j in range(1,11):
                images['DOG'][s][rt][j] = {}
                for k in range(1,11):
                    if(k%2 == 0 and j %2 == 0) or (k%2 != 0 and j%2 == 0) or (j%2 != 0 and k%2 == 0):
                        pass
                    else:
                        temp1 = cv.GaussianBlur(i,(j,j),1)
                        temp2 = cv.GaussianBlur(i,(k,k),1.6)
                        result = temp1 - temp2
                        images['DOG'][s][rt][j][k] = result
                        
    elif s == 2:
        images['DOG'][s] = {}
        for i in gray_images:
            rt = rt+1
            images['DOG'][s][rt] = {}
            for j in range(1,11):
                images['DOG'][s][rt][j] = {}
                for k in range(1,11):
                    if(k%2 == 0 and j %2 == 0) or (k%2 != 0 and j%2 == 0) or (j%2 != 0 and k%2 == 0):
                        pass
                    else:
                        temp1 = cv.GaussianBlur(i,(j,j),10)
                        temp2 = cv.GaussianBlur(i,(k,k),16)
                        result = temp1 - temp2
                        images['DOG'][s][rt][j][k] = result

In [ ]:
#Storing Grayscale images in images dictionary
#images['Gray']['Image number (1-100)']
rt = 0
for i in gray_images:
    rt = rt + 1
    images['Gray'][rt] = i
    
rt = 0
for i in dcm_equalized:
    rt = rt + 1
    images['Gray_Equalized'][rt] = i